In [8]:
%pip install langchain_community
%pip install langchain
%pip install chromadb
%pip install langchain_text_splitters

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install -q -U google-generativeai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Generating Vector DB

In [1]:
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

DATA_PATH = r"RAG DATA"
CHROMA_PATH = r"VECTOR DATABASE"

if not os.path.exists(CHROMA_PATH):
    os.makedirs(CHROMA_PATH)

os.chmod(CHROMA_PATH, 0o777)

chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)

collection_name = "psychologist"
try:
    chroma_client.delete_collection(name=collection_name)
except Exception:
    print("No existing collection to delete.")
collection = chroma_client.get_or_create_collection(name=collection_name)

loader = PyPDFDirectoryLoader(DATA_PATH)
raw_documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(raw_documents)

documents = [chunk.page_content for chunk in chunks]
ids = [f"ID{i}" for i, _ in enumerate(chunks)]
metadata = [chunk.metadata for chunk in chunks]

collection.upsert(
    documents=documents,
    metadatas=metadata,
    ids=ids,
)

print("Data successfully added to ChromaDB.")


No existing collection to delete.
Data successfully added to ChromaDB.


# Example Query Output

## Checking llm working

In [3]:
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("Google_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("what is the best way to grow tomatoes?")
print(response.text)

There's no single "best" way, as the ideal method depends on your climate, resources, and personal preferences. However, here's a breakdown of best practices for growing healthy, delicious tomatoes:

**1. Choosing the Right Variety:**

* **Consider your climate:**  Choose varieties suited to your region's growing season.  Indeterminate tomatoes (vining, grow continuously) are best for long, warm seasons, while determinate tomatoes (bushy, produce fruit all at once) are better for shorter seasons.
* **Disease resistance:** Select varieties resistant to common diseases in your area, such as blight, fusarium wilt, and verticillium wilt.
* **Taste preference:**  Explore different types – heirloom, hybrid, cherry, beefsteak, etc. – to find your favorites.

**2. Starting Seeds (or Buying Seedlings):**

* **Starting from seed:** This gives you more control and a wider variety selection. Start seeds indoors 6-8 weeks before the last expected frost. Use seed-starting mix, not garden soil.
* **B

## Checking Rag working

In [4]:
import chromadb
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()

CHROMA_PATH = r"VECTOR DATABASE"
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = chroma_client.get_or_create_collection(name="psychologist")

user_query = input("How are you feeling today? I'm here to listen.\n\n")

results = collection.query(
    query_texts=[user_query],
    n_results=1
)

genai.configure(api_key=os.getenv("Google_API_KEY"))

model = genai.GenerativeModel(
    'gemini-1.5-flash',
    system_instruction=f"""
    You are a compassionate and deeply empathetic psychologist. Your goal is to provide a safe space where the user feels heard, understood, and supported. 
    Respond in a warm and non-judgmental manner, offering emotional validation and gentle guidance.
    
    Always prioritize active listening, use open-ended questions to encourage reflection, and offer comfort when needed.
    If relevant, provide simple mindfulness or grounding techniques to help the user feel more at ease.
    
    Strictly answer ONLY based on the provided context.
    Context: {results['documents']}
    """
)

response = model.generate_content(user_query)

print("\n\n---------------------\n\n")
print(response.text)




---------------------


Hello.  I hear you saying you're feeling depressed, and it takes a lot of courage to share that.  Thank you for trusting me enough to talk about it.  It sounds like you're going through something difficult right now.  

Can you tell me a little more about what's been weighing on you lately?  There's no pressure to share anything you're not comfortable with, but talking about it can sometimes help.  Even just describing how you're feeling physically and emotionally could be a good starting point.

In the meantime, let's try a simple grounding technique.  Notice five things you can see around you, four things you can touch, three things you can hear, two things you can smell, and one thing you can taste.  This can sometimes help to bring you back to the present moment.

I'm here to listen without judgment, and I want you to know you're not alone.

